# How to build an AI Agent with OpenAI and the ClickHouse MCP Server

In this notebook we'll see how to build an [Open AI](https://github.com/openai/openai-agents-python) AI agent that can interact with [ClickHouse's SQL playground](https://sql.clickhouse.com/) using [ClickHouse's MCP Server](https://github.com/ClickHouse/mcp-clickhouse).


## Install libraries
We need to install the Agno library.

In [ ]:
!pip install -q --upgrade pip

In [2]:
!pip install -q openai-agents

## Setup credentials
Let's provide our Anthropic API key.

In [4]:
import os, getpass

In [15]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OpenAI API Key:")

Enter OpenAI API Key: ········


We'll also define the credentials to connect to the ClickHouse SQL playground:

In [11]:
env = {
    "CLICKHOUSE_HOST": "sql-clickhouse.clickhouse.com",
    "CLICKHOUSE_PORT": "8443",
    "CLICKHOUSE_USER": "demo",
    "CLICKHOUSE_PASSWORD": "",
    "CLICKHOUSE_SECURE": "true"
}

## Initialize MCP Server and OpenAI agent

Lets configure the ClickHouse MCP Server to point at the ClickHouse SQL playground and also initialize our OpenAI agent and ask it a question:

In [47]:
from agents.mcp import MCPServer, MCPServerStdio
from agents import Agent, Runner, trace
import json

In [49]:
def simple_render_chunk(chunk):
    """Simple version that just filters important events"""
    
    # Tool calls
    if (hasattr(chunk, 'type') and 
        chunk.type == 'run_item_stream_event'):
        
        if chunk.name == 'tool_called':
            tool_name = chunk.item.raw_item.name
            args = chunk.item.raw_item.arguments
            print(f"🔧 Tool: {tool_name}({args})")
        
        elif chunk.name == 'tool_output':
            try:
                # Handle both string and already-parsed output
                if isinstance(chunk.item.output, str):
                    output = json.loads(chunk.item.output)
                else:
                    output = chunk.item.output
                
                # Handle both dict and list formats
                if isinstance(output, dict):
                    if output.get('type') == 'text':
                        text = output['text']
                        if 'Error' in text:
                            print(f"❌ Error: {text}")
                        else:
                            print(f"✅ Result: {text[:100]}...")
                elif isinstance(output, list) and len(output) > 0:
                    # Handle list format
                    first_item = output[0]
                    if isinstance(first_item, dict) and first_item.get('type') == 'text':
                        text = first_item['text']
                        if 'Error' in text:
                            print(f"❌ Error: {text}")
                        else:
                            print(f"✅ Result: {text[:100]}...")
                else:
                    # Fallback - just print the raw output
                    print(f"✅ Result: {str(output)[:100]}...")
                    
            except (json.JSONDecodeError, AttributeError, KeyError) as e:
                # Fallback to raw output if parsing fails
                print(f"✅ Result: {str(chunk.item.output)[:100]}...")
        
        elif chunk.name == 'message_output_created':
            try:
                content = chunk.item.raw_item.content
                if content and len(content) > 0:
                    print(f"💬 Response: {content[0].text}")
            except (AttributeError, IndexError):
                print(f"💬 Response: {str(chunk.item)[:100]}...")
    
    # Text deltas for streaming
    elif (hasattr(chunk, 'type') and 
          chunk.type == 'raw_response_event' and
          hasattr(chunk, 'data') and 
          hasattr(chunk.data, 'type') and
          chunk.data.type == 'response.output_text.delta'):
        print(chunk.data.delta, end='', flush=True)

In [50]:
async with MCPServerStdio(
        name="ClickHouse SQL Playground",
        params={
            "command": "uv",
            "args": [
                'run',
                '--with', 'mcp-clickhouse',
                '--python', '3.13',
                'mcp-clickhouse'
            ],
            "env": env
        }, client_session_timeout_seconds = 60
    ) as server:
        agent = Agent(
            name="Assistant",
            instructions="Use the tools to query ClickHouse and answer questions based on those files.",
            mcp_servers=[server],
        )

        message = "What's the biggest GitHub project so far in 2025?"
        print(f"\n\nRunning: {message}")
        with trace("Biggest project workflow"): 
            result = Runner.run_streamed(starting_agent=agent, input=message, max_turns=20)
            async for chunk in result.stream_events():
                simple_render_chunk(chunk)



Running: What's the biggest GitHub project so far in 2025?
🔧 Tool: list_databases({})
✅ Result: amazon
bluesky
country
covid
default
dns
environmental
food
forex
geo
git
github
hackernews
imdb
log...
🔧 Tool: list_tables({"database":"github"})
✅ Result: {
  "database": "github",
  "name": "actors_per_repo",
  "comment": "",
  "columns": [
    {
      "...
🔧 Tool: run_select_query({"query":"SELECT repo_name, MAX(stars) FROM github.top_repos_mv"})
✅ Result: {
  "status": "error",
  "message": "Query failed: HTTPDriver for https://sql-clickhouse.clickhouse....
🔧 Tool: run_select_query({"query":"SELECT repo_name, stars FROM github.top_repos ORDER BY stars DESC LIMIT 1"})
✅ Result: {
  "repo_name": "sindresorhus/awesome",
  "stars": 402893
}...
The biggest GitHub project in 2025, based on stars, is "[sindresorhus/awesome](https://github.com/sindresorhus/awesome)" with 402,893 stars.💬 Response: The biggest GitHub project in 2025, based on stars, is "[sindresorhus/awesome](https://github.com